### Use Machine Learning to predict the probability of needing Post-Mastectomy Radiotherapy
Author: Yifu (Charles) Chen


In [ ]:
### Import modules ###
import os
import glob
import pandas as pd
import numpy as np
from typing import Dict
import re
import copy
# Machine Learning Modules for Random Forest, Logistic Regression, and XGBoost
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
# Metrics
from sklearn import metrics
# Logging
import logging
import sys

### Utils

In [21]:
# Setup no print limits and etc.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('expand_frame_repr', False)

class bcolors:
    # Helper class to print in terminal with colors
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    NORMAL = '\033[0m'
    
def setup_logging():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.NOTSET)
    logging.basicConfig(
            format="%(asctime)s - %(levelname)s - %(message)s \n",
            datefmt="%m/%d/%Y %H:%M:%S",
            handlers=[logging.StreamHandler(sys.stdout), logging.FileHandler(f"{os.getcwd()}/log.txt")],
            level=logging.NOTSET
        )
    return logger

logger = setup_logging()

In [22]:
from collections import OrderedDict

# Directory of DataFrames
dir_to_df = "/Users/yifu/PycharmProjects/Radiotherapy-Prediction/data/output/2022-07-13-133052_without_imputation/DataFrames/subset_dataframes"

# Path to log file
log_path = os.path.join(dir_to_df, "log.txt")
use_full_cols = True
# Create a new log file from scratch, regardless if one exists
with open(log_path, "w") as log_file:
    log_file.write("")

def print_and_log(*args):
    # Helper function to print and log
    # Can handle writing DataFrame to log file
    logger.info(*args)
    with open(log_path, "a") as log_file:
        if type(args[0]) == pd.DataFrame:
            log_file.write(args[0].to_string() + "\n")
        else:
            log_file.write("|| ".join([str(a) for a in args]) + "\n")

    
# Assume all files in directory are CSV DataFrame files
paths_to_df = glob.glob(os.path.join(dir_to_df, "*.csv"))

# Read each DataFrame file
dfs = OrderedDict()
for i, path in enumerate(paths_to_df):
    if use_full_cols and "1.0" not in path:
        continue
    df_name = path.replace(dir_to_df, "")
    df_name = df_name.replace("/", "").replace(".csv", "")
    df = pd.read_csv(path)
    if i == len(paths_to_df) - 1:
        print("Kept the largest DF")
        df_all = df.copy()
        # dfs["df_all"] = df
    # Drop "PRE_record_id" column, if it exists
    if "PRE_record_id" in df.columns:
        df = df.drop(columns=["PRE_record_id"], axis=1)
    dfs[df_name] = df
# Sorted the dict by path name
dfs = OrderedDict(sorted(dfs.items()))

for k in dfs.keys():
    print_and_log(k)
    



Kept the largest DF
07/26/2022 14:24:06 - INFO - 9_PRE-1.0spars-expert-ML-imputed89cols 

07/26/2022 14:24:06 - INFO - E_POS-1.0spars-expert-ML-imputed132cols 



### Pre-Process Datasets

In [24]:
# Standardize the data by removing the mean and scaling to unit variance
# The target variable is named target_column

dfs_std = OrderedDict()

for df_name, df in dfs.items():
    # Standardize all columns except the target column target_column
    df_std = df.copy()
    df_std.drop([target_column], axis=1, inplace=True)
    df_std = (df_std - df_std.mean()) / df_std.std()
    df_std = df_std.copy()
    # Concatenate the target column back
    df_std[target_column] = df[target_column]
    # Convert the target column to binary
    df_std[target_column] = df_std[target_column] > 0
    dfs_std[df_name] = df_std

len(dfs_std)

KeyError: "['POS_metastasis'] not found in axis"

### Build Machine Learning Classifiers

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# Impute missing values using sklearn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

@ignore_warnings(category=ConvergenceWarning)
def impute_missing_value(df_unimputed, target_column, max_iter):
    """Given a DataFrame with missing values, impute them using IterativeImputer.
    Leave the target_column as is.

    Args:
        df_unimputed (DataFrame): DataFrame with missing values.
    """
    imp = IterativeImputer(max_iter=max_iter, random_state=0, verbose=0)
    X = df_unimputed.drop(target_column, axis=1)
    y = df_unimputed[target_column]
    X_imputed = imp.fit_transform(X)
    df_imputed = pd.DataFrame(X_imputed, columns=X.columns)
    df_imputed[target_column] = y
    return df_imputed

def standardize_data(df_unstandardized, target_column):
    """Standardize the data using sklearn.

    Args:
        df_unstandardized (DataFrame): DataFrame with missing values.
    """
    scaler = StandardScaler()
    X = df_unstandardized.drop(target_column, axis=1)
    y = df_unstandardized[target_column]
    X_standardized = scaler.fit_transform(X)
    df_standardized = pd.DataFrame(X_standardized, columns=X.columns)
    df_standardized[target_column] = y
    # Drop columns with all NaNs
    df_standardized.dropna(axis=1, how="all", inplace=True)
    return df_standardized
    

In [ ]:


# Initialize a dictionary for holding the models for each DataFrame
# Schema: {"DataFrame Name": {"Model Name": Model Object, ...}, ...}
from collections import defaultdict
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import f1_score
import itertools

all_models = defaultdict(dict)

# The print_df contains the dataframe, model, and the average CV accuracy, AUC, and the top 6 features and their scores
# metrics_df_columns = ["Dataset", "Model", "Avg CV Accuracy", "Avg CV AUC", "Avg CV F1", "Feature_1", "Score_1", "Feature_2", "Score_2", 
#                     "Feature_3", "Score_3", "Feature_4", "Score_4", "Feature_5", "Score_5", "Feature_6", "Score_6"]

metrics_df_columns = ["Dataset", "Model"] + list(itertools.chain(*[["Avg " + m, "SE " + m] for m in ["Accuracy", "AUC", "F1"]]))

def build_classifiers(df: pd.DataFrame, df_name: str, num_folds: int, target_col: str) -> Dict[str, object]:
    """
    For each DataFrame, build the following classifiers:
        Logistic Regression
        Random Forest
        XGBoost
    Args:
        df (pd.DataFrame): the DataFrame to build the classifiers for
        df_name (str): the name of the DataFrame
        num_folds (int): the number of folds to use for cross-validation
        target_col (str): the name of the target column
    Returns:
        None
    """
    # # Impute missing values with KNN
    # from sklearn.impute import KNNImputer
    # imputer = KNNImputer(n_neighbors=5)
    # # Drop columns that are all NaN, also drop the target column
    # df_drop_na = df.dropna(axis=1, how="all")
    # df_drop_na = df_drop_na.drop(columns=[target_col])
    # # Impute the missing values
    # df_imp = pd.DataFrame(imputer.fit_transform(df_drop_na), columns=df_drop_na.columns)
    # # Put the target back
    # df_imp[target_col] = df[target_col]
    # df = df_imp.copy()
    df = standardize_data(df, target_col)
    df = impute_missing_value(df, target_col, max_iter=20)
    assert df.isnull().sum().sum() == 0

    # Initialize a dictionary for holding the models for each DataFrame
    # Schema: {"Fold 1": {"Model Name": Model Object, ...}, ...}
    models = defaultdict(dict)  
    performance = defaultdict(dict)
    # Train each model using 5-fold cross-validation, repeated 10 times to get error bars
    # for i in range(num_cvs):
    kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1)
    for fold, (train_index, test_index) in enumerate(kf.split(df, df[target_col])):
        fold = fold + 1
        print("Fold {}".format(fold), end="... " if fold < num_folds else "\n")
        # Split the DataFrame into training and validation sets
        df_train, df_val = df.iloc[train_index], df.iloc[test_index]
        # Build the classifiers
        models[fold] = {"Logistic Regression": LogisticRegression(),
                        "Random Forest": RandomForestClassifier(),
                        "XGBoost": XGBClassifier()
                        }
        for model_name, model in models[fold].items():
            # Train each model on the target column POS_did_the_patient_receive_pm
            # X (all variables for training), Y (target)
            model.fit(
                df_train.drop([target_col], axis=1),
                df_train[target_col]
                )
            # Predict the validation set
            y_pred = model.predict(
                df_val.drop([target_col], axis=1)
            )
            # Calculate the accuracy of the model
            accuracy = np.mean(y_pred == df_val[target_col])
            # Calcuilate the F1
            F1 = f1_score(y_true=df_val[target_col], y_pred=y_pred)
            # Predict the probability of the validation set
            y_pred_proba = model.predict_proba(df_val.drop([target_col], axis=1))[:, 1]
            # Calculate the AUC of the model
            auc = metrics.roc_auc_score(df_val[target_col], y_pred_proba)
            if model_name != "Logistic Regression":
                # Get the features for the model
                feature_importances = pd.Series(model.feature_importances_, index=df_train.drop([target_col], axis=1).columns)
            else:
                # The features for the Logistic Regression model are the coefficients
                # Take absolute value of the coefficients
                feature_importances = pd.Series(np.abs(model.coef_[0]), index=df_train.drop([target_col], axis=1).columns)
            # Store the accuracy, AUC, and features for the model
            performance[model_name][fold] = {"accuracy": accuracy, "auc": auc, "F1": F1, "top_features": feature_importances}
    
    # Consolidate the top features for each model
    # Schema: {"Model Name": {"Feature_1": Score_1, ...}, ...}
    feature_scores = defaultdict(lambda: defaultdict(float))
    for model_name, fold in performance.items():
        for fold_num, fold_performance in fold.items():
            # Extract feature name and importance score
            for feature_name, score in fold_performance["top_features"].items():
                feature_scores[model_name][feature_name] += score
        # Divide the feature importance scores by the number of folds
        for feature_name, score in feature_scores[model_name].items():
            feature_scores[model_name][feature_name] /= len(fold)
            # Round to 4 decimal places
            feature_scores[model_name][feature_name] = round(feature_scores[model_name][feature_name], 4)
    
    # Print the performance of each model, including averaged across folds
    # DataFrame for printing results in a nice format
    # Columns: Dataset, Model, Avg CV Accuracy, SE Accuracy, Avg CV AUC, SE AUC, Avg CV F1, SE F1
    print_df = pd.DataFrame(columns=metrics_df_columns)
    for model_name, fold in performance.items():
        accuracy = round(np.mean([fold[fold_num]["accuracy"] for fold_num in fold.keys()]), 4)
        auc = round(np.mean([fold[fold_num]["auc"] for fold_num in fold.keys()]), 4)
        F1 = round(np.mean([fold[fold_num]["F1"] for fold_num in fold.keys()]), 4)
        accuracy_se = round(np.std([fold[fold_num]["accuracy"] for fold_num in fold.keys()]) / np.sqrt(len(fold)), 4)
        auc_se = round(np.std([fold[fold_num]["auc"] for fold_num in fold.keys()]) / np.sqrt(len(fold)), 4)
        F1_se = round(np.std([fold[fold_num]["F1"] for fold_num in fold.keys()]) / np.sqrt(len(fold)), 4)
        curr_df = pd.DataFrame(
                    [
                        [df_name, model_name, accuracy, accuracy_se, auc, auc_se, F1, F1_se]
                    ],
                    columns=metrics_df_columns
                )
        print_df = pd.concat(
            [
                print_df,
                curr_df
            ],
            axis=0
        )
    # Sort feature scores, using OrderedDict
    for model_name, model_feature_scores in feature_scores.items():
        feature_scores[model_name] = OrderedDict(sorted(model_feature_scores.items(), key=lambda x: x[1], reverse=True))
    
    return print_df, feature_scores

def generate_feature_scores_and_plot(dfs_std, target_col):
    """Generate feature scores and plots for each data subset.

    Args:
        dfs_std: a dictionary with standardized dataframes for each data subset

    Returns:
        plot_dict, all_top_features: a dictionary with plots for each data subset, and a dictionary with all top features for each model
    """
    # Build the classifiers for each DataFrame
    print_df = pd.DataFrame(columns=metrics_df_columns)
    # Create a dictionary for holding the DataFrames for plotting
    plot_dict = OrderedDict()
    # Create a dictionary for holding all the feature scores across datasets and models
    # Schema: {"Dataset1": {"Model1": {"Feature_1": Score_1, ...}, ...}, ...}
    all_top_features = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
    for df_name, df in dfs_std.items():
        print_and_log(f"Processing {df_name}...")
        print_df, feature_scores = build_classifiers(df, df_name, num_folds=10, target_col=target_col)
        # print the DataFrame without the index
        print_and_log(print_df.to_string(index=False))
        model_names = feature_scores.keys()
        for model in model_names:
            top_features = sorted(feature_scores[model].items(), key=lambda x: x[1], reverse=True)[:]
            print_and_log(
                "\n" + "-" * 10 + 
                f"The 1/3 ({len(top_features)//3} out of {len(top_features)}) "
                f"most important features for {model}:" + "-" * 10
            )
            for i, (feat, score) in enumerate(top_features):
                all_top_features[df_name][model][feat] = score
                # If the feature is in the top 1/3, print it
                if i < len(top_features) / 3:
                    print_and_log(f"{feat}: {score}")
        print_and_log("="*150)
        # Add the DataFrame to the dictionary for plotting later
        plot_dict[df_name] = print_df
    return plot_dict, all_top_features

plot_dict, all_top_features = generate_feature_scores_and_plot(dfs_std, target_column)

07/26/2022 14:23:03 - INFO - Processing 9_PRE-1.0spars-expert-ML-imputed89cols... 



/Users/yifu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:980: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/yifu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/Users/yifu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:1005: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Fold 1... 

/Users/yifu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 2... 

...

Fold 10


/Users/yifu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


07/26/2022 14:23:44 - INFO -                                Dataset               Model Avg Accuracy SE Accuracy Avg AUC  SE AUC  Avg F1   SE F1
9_PRE-1.0spars-expert-ML-imputed89cols Logistic Regression       0.7069      0.0115  0.7657  0.0126   0.615  0.0174
9_PRE-1.0spars-expert-ML-imputed89cols       Random Forest       0.7245      0.0162  0.7712  0.0181  0.6208  0.0275
9_PRE-1.0spars-expert-ML-imputed89cols             XGBoost       0.7008      0.0135  0.7599  0.0176  0.6048  0.0236 

07/26/2022 14:23:44 - INFO - 
----------The 1/3 (28 out of 84) most important features for Logistic Regression:---------- 

07/26/2022 14:23:44 - INFO - PRE_systhe___radiation: 1.1317 

07/26/2022 14:23:44 - INFO - PRE_num_closest_margins_trans: 0.8031 

07/26/2022 14:23:44 - INFO - PRE_bra_cup_size: 0.6574 

07/26/2022 14:23:44 - INFO - PRE_tumor_max_size_composite: 0.6182 

07/26/2022 14:23:44 - INFO - PRE_int_mammary_lymphade_pet: 0.4823 

07/26/2022 14:23:44 - INFO - PRE_lymph_node_max_size_mm: 0

/Users/yifu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:980: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/yifu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/Users/yifu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:1005: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


KeyboardInterrupt: 

### Plotting

In [ ]:
plot_dir = os.path.join(dir_to_df, "plots")
# Create the directory if it doesn't exist
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

: 

In [ ]:
# Import plotting libraries for machine learning experiments
import matplotlib.pyplot as plt
import seaborn as sns



# Retrive the saved DataFrames, and plot the results
# Plot the results for each DataFrame on a single plot
# Reset plot parameters
plt.rcParams.update(plt.rcParamsDefault)
for use_broken_axis in [False, True]:
    for metric_name in ["Accuracy", "AUC", "F1"]:
        y_data = []
        errors = []
        x_data = []
        # Retrieve the performance for each model
        for df_name, print_df in plot_dict.items():
            # Iterate through the models of the DataFrame
            for i, row in print_df.iterrows():
                # If the model is the metric, add it to the list
                model_name = row["Model"]
                # Use regular expression to retrieve the metric and its standard error
                metric = row["Avg " + metric_name]
                error = row["SE " + metric_name]
                df_name = row["Dataset"]
                df_name = df_name.replace("/", "").replace(".csv", "")
                x_data.append(metric)
                y_data.append(df_name + " - " + model_name)
                errors.append(error)
        # Create the plot
        # The plot has spaces between the bars
        plt.figure(figsize=(30, 16))
        # Plot the gridlines
        sns.set_style("whitegrid")
        # The color of the bar should alternate between 3 colors
        colors = ["#1f77b4", "#ff7f0e", "#2ca02c"]
        # Make bars
        plt.barh(y_data, x_data, xerr=errors, color=colors, align="center", alpha=0.7)
        plt.yticks(fontsize=18)
        plt.xticks(fontsize=18)
        # Align the y-tick texts to the left
        ax = plt.gca()
        # for tick in ax.yaxis.get_majorticklabels():
        #     tick.set_horizontalalignment("left")
        # Set the labels on top of the bars
        for i, v in enumerate(x_data):
            plt.text(v + 0.05, y_data[i], str(v), color="black", fontsize=18)
        plt.gca().invert_yaxis()
        if use_broken_axis:
            # Set the range of the x-axis to be between the min - 0.1 and 1
            axis_start = min(x_data) - 0.1
            plt.xlim(axis_start, 1)

        plt.xlabel(metric_name, fontsize=20)
        plt.ylabel("Model")
        plt.title(f"{metric_name} by Model", fontsize=40)

        plt.tight_layout()
        if use_broken_axis:
            # Save the plot
            plt_name = f"{metric_name}_br_axis.png"
        else:
            plt_name = f"{metric_name}_full_axis.png"
        plt.savefig(os.path.join(plot_dir, plt_name))
        plt.show()
        plt.close()



: 

In [ ]:
# For each dataset in all_top_features, build a DataFrame with the top features
# The features for each model is ranked by their score
# The most important features shared by all models are sorted to the top
# Create a dictionary for holding the feature scores per dataset
# Schema: {"Dataset1": {"Model1": {"Feature_1": score_1, ...}, ...}, ...}
def tabularize_feature_ranks(all_top_features, model_names, merged_df_prefixes, top_k_percent=10, do_print=True):
    """Given a dictionary of top features, tabulate the feature ranks for each model

    Args:
        all_top_features (_type_): A dictionary of top features
        model_names (list, optional): Models for the merged table. Defaults to ["Random Forest"].
        merged_df_prefixes (list, optional): Prefixes for the merged dataset names, e.g., ["0", "1", ...]
        top_k_percent (int, optional): The top k percent of features to be counted in the final table. Defaults to 40.
        do_print (bool, optional): Whether to print the table. Defaults to True.

    """
    # Keep only the datasets that startswith prefixes in merged_df_prefixes
    # Create a copy of all_top_features
    all_top_features = copy.deepcopy(all_top_features)
    for df_name in list(all_top_features.keys()):
        if not any([df_name.startswith(prefix) for prefix in merged_df_prefixes]):
            del all_top_features[df_name]
    
    result_df_dict = OrderedDict()
    # Columns: Dataset1 Score, Dataset2 Score, ...
    # Rows: Feature1, Feature2, ...
    # We currently only consider the Random Forest model
    for dataset, model_dict in all_top_features.items():
        if do_print:
            print_and_log("-"*50)
            print_and_log(f"Processing {dataset}...")
            # Print the number of features for this dataset
            print_and_log(f"Number of features: {len(model_dict['Random Forest'])}")
        LR_features = model_dict["Logistic Regression"]
        RF_features = model_dict["Random Forest"]
        XGB_features = model_dict["XGBoost"]
        # Sort the features by their scores
        sorted_LR_features = sorted(LR_features.items(), key=lambda x: x[1], reverse=True)
        sorted_RF_features = sorted(RF_features.items(), key=lambda x: x[1], reverse=True)
        sorted_XGB_features = sorted(XGB_features.items(), key=lambda x: x[1], reverse=True)

        # Convert each score into a rank
        rank_LR_features = {feat: i+1 for i, (feat, score) in enumerate(sorted_LR_features)}
        rank_RF_features = {feat: i+1 for i, (feat, score) in enumerate(sorted_RF_features)}
        rank_XGB_features = {feat: i+1 for i, (feat, score) in enumerate(sorted_XGB_features)}
        # Assert all the features are present in the sorted features dictionary
        assert set(rank_LR_features.keys()) == set(rank_RF_features.keys()) == set(rank_XGB_features.keys())
        # Create a DataFrame with all the features
        df = pd.DataFrame({"Features": list(rank_LR_features.keys())})
        # Calculate the feature scores for each model
        # Add the ranks of RF to the DataFrame
        df["RF_rank"] = df["Features"].map(rank_RF_features)
        # Add the ranks of LR to the DataFrame
        df["LR_rank"] = df["Features"].map(rank_LR_features)
        # Add the ranks of XGB to the DataFrame
        df["XGB_rank"] = df["Features"].map(rank_XGB_features)
        # Add the raw scores of RF to the DataFrame
        df["RF_score"] = df["Features"].map(RF_features)
        # Add the raw scores of LR to the DataFrame
        df["LR_score"] = df["Features"].map(LR_features)
        # Add the raw scores of XGB to the DataFrame
        df["XGB_score"] = df["Features"].map(XGB_features)
        # Add a column corresponding to the overall ranking using Rank Product Statistic (geometric mean of the ranks)
        df["Rank_Product^(1/3)"] = (df["RF_rank"] * df["LR_rank"] * df["XGB_rank"]) ** (1/3)
        # Sort the DataFrame by the ranks of the features
        df = df.sort_values(by="RF_rank")
        df_rank_product = df.sort_values(by="Rank_Product^(1/3)")
        # Reset the index
        df = df.reset_index(drop=True)
        # Add Rank_Product_Rank column, where the rank is the index of the row in df_rank_product
        product_rank_map = {feat: i+1 for i, feat in enumerate(df_rank_product["Features"])}
        df["Rank_Product_Rank"] =  df["Features"].map(product_rank_map)
        
        # Keep only the top 1/3 of features (the most important features)
        # df = df.iloc[:len(df)//3]
        # Round all the entries to 2 decimal places
        df = df.round(2)
        result_df_dict[dataset] = df
        # Print the DataFrame, ignoring the index
        if do_print:
            print_and_log(df.to_string(index=False))
    
    # # Keep only the datasets that startswith prefixes in merged_df_prefixes
    # result_df_dict = {k: v for k, v in result_df_dict.items() if k.startswith(tuple(merged_df_prefixes))}
    
    # Get all the features from the feature importance 
    all_features = set()
    for dataset, df in result_df_dict.items():
        all_features.update(df["Features"].values)


    # Merge the DataFrames for each dataset
    # The rows are the features, the columns are the datasets
    # The values are the Random Forest and Logistic Regression scores of the features in each dataset
    merged_df = pd.DataFrame({"Features": list(all_features)})
    # Merge feature scores
    for dataset_name, df in result_df_dict.items():
        dataset_name = dataset_name.replace("expert", "").replace("imputed", "")
        dataset_name = re.sub("-{2,}", "-", dataset_name)
        # Keep only the columns RF_score and LR_score
        for model_name in model_names:
            col_name = model_name + "_" + dataset_name
            feat_to_score_dict = {feat: df[df["Features"] == feat][model_name + "_score"].values[0] for feat in df["Features"]}
            mean_feature_score = np.mean(list(feat_to_score_dict.values()))
            for i, row in merged_df.iterrows():
                feat = row["Features"]
                if feat in feat_to_score_dict:
                    merged_df.loc[i, col_name] = feat_to_score_dict[feat]
                else:
                    merged_df.loc[i, col_name] = np.nan
    # Create a column that counts the frequency of the feature score being among the top features
    # For each col in rf_score_cols, and each feature, calculate the frequency of the feature score being among the top K% in this col
    top_k_freq_dict = defaultdict(lambda: defaultdict(int))
    # Schema: {model_name: {feature: int}}
    for dataset_name, df in result_df_dict.items():
        df_sorted = df.sort_values(by="RF_rank")
        dataset_name = dataset_name.replace("expert", "").replace("imputed", "")
        dataset_name = re.sub("-{2,}", "-", dataset_name)
        for model_name in model_names:
            col_name = model_name + "_" + dataset_name
            for i, row in merged_df.iterrows():
                feat = row["Features"]
                feat_score = row[col_name]
                is_top_k = 0
                if feat_score in df_sorted.iloc[:max(1, int(len(df)*top_k_percent)//100)][model_name+"_score"].values:
                    is_top_k = 1
                top_k_freq_dict[model_name][feat] += is_top_k
                # if feat == "PRE_age_at_dx" and is_top_k:
                #     print(f"age_at_dx score: {feat_score} in top K of {dataset_name} {model_name}")
                # raise ValueError("Fix this problem, not sure why LR isn't counted correctly")
    
    # Insert Top_K%_Freq columns after "Features" column
    for i, model_name in enumerate(model_names):
        merged_df.insert(
            i+1, model_name+ f"_Top_{top_k_percent}%_Freq",
            merged_df["Features"].map(top_k_freq_dict[model_name])
            )

    # Insert _Top_Probability columns
    # Where the value is the probability it's top-K% vs the number of times the feature occurred in dataset
    num_feature_occurred = defaultdict(int)
    for df_name, model_dict in all_top_features.items():
        for model, feat_dict in model_dict.items():
            for feat in feat_dict.keys():
                num_feature_occurred[feat] += 1
            break
    
    for i, model_name in enumerate(model_names):
        merged_df.insert(
            i+1, model_name+f"_Top_{top_k_percent}%_Prob",
            merged_df["Features"].map(lambda feat: top_k_freq_dict[model_name][feat]/num_feature_occurred[feat] if feat in feat_dict else 0)
            )
    # Sort the DataFrame by Top_K%_Probability
    merged_df = merged_df.sort_values(
        by=[model_name + f"_Top_{top_k_percent}%_Prob" for model_name in model_names] + \
           [model_name + f"_Top_{top_k_percent}%_Freq" for model_name in model_names],
        ascending=False)

    # Cleaning up the merged_df
    # Round all the entries to 2 decimal places
    merged_df = merged_df.round(2)
    # Remove rows with NaN in all except the column "Features", i.e. the feature was not used by any model
    merged_df = merged_df.dropna(how="all", subset=merged_df.columns[1:])
    
    print_and_log(f"Total number of features: {len(merged_df)}")

    # Save the merged DataFrame
    merged_df_path = os.path.join(plot_dir, f"merged_feature_scores_datasets{''.join(merged_df_prefixes)}.csv")
    merged_df.to_csv(merged_df_path, index=False)
    print_and_log(f"Merged DataFrame saved to {merged_df_path}")
    return merged_df, top_k_freq_dict

df_merged, top_k_freq_dict = tabularize_feature_ranks(
    all_top_features,
    model_names=["RF", "LR"],
    merged_df_prefixes=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
    do_print=False
)

df_merged, top_k_freq_dict = tabularize_feature_ranks(
    all_top_features,
    model_names=["RF", "LR"],
    merged_df_prefixes=[df_name[0] for df_name in all_top_features.keys()],
    do_print=False
)
# print_and_log(df_merged.to_string(index=False))

: 

In [ ]:
# Conduct experiments for determining the cumulative effect of the features on the model performance
# For each dataset, train the models on the top-1 most important feature, then top-2, top-3, ...
# Save the results of each experiment in a dictionary
# Import sleep module to allow the program to pause for a few seconds
import time
# def ... TODO: wrap the following code into a neat, reusable function
num_folds = 50
model_name = "Random Forest"
target_column = "POS_did_the_patient_receive_pm"
metrics_df = pd.DataFrame(columns=metrics_df_columns)
# Create a dictionary for holding the DataFrames for plotting
# Schema: {"Dataset1": {"Top1_feature": metrics_df1, "Top2_feature": metrics_df2, ...}, ...}
plot_dict2 = defaultdict(OrderedDict)
# Re-use the all_top_features dictionary for deciding the order of the features to use

def plot_num_features_vs_metrics(all_top_features, model_name, target_column, metrics_df, plot_dict2, plot_dir):
    
    for df_name in all_top_features:
        if not ("1.0" in df_name):
            continue
        # Create a DataFrame for the top-k most important feature
        # Sort all features by feature scores
        all_features_sorted = [f[0] for f in sorted(all_top_features[df_name][model_name].items(), key=lambda x: x[1], reverse=True)]
        num_features = len(all_features_sorted)
        df_all_features = dfs_std[df_name]
        for k in range(0, num_features+1):
            if k == 0:
                # Random noise baseline
                top_k_features = "random_noise_baseline"
                print_and_log(f"Processing {df_name} with top-{k} features out of {num_features}. This feature is random noise baseline.")
                df_top_k_features = df_all_features[[target_column]]
                noise_data = np.random.rand(len(df_top_k_features))
                df_top_k_features.insert(0, "0_random_noise_baseline", noise_data)
                assert len(df_top_k_features.columns) == 2
            else:
                # Get the top-k most important features
                top_k_features = all_features_sorted[:k]
                print_and_log(f"Processing {df_name} with top-{k} features out of {num_features}. The k-th is {top_k_features[-1]}")
                # Create a DataFrame for the top-k most important features, and the target variable
                df_top_k_features = df_all_features[top_k_features + [target_column]]
                assert len(df_top_k_features.columns) == k + 1
            metrics_df, feature_scores = build_classifiers(df_top_k_features, df_name, num_folds=num_folds)
            # print metrics_df, without index
            print_and_log("")
            print_and_log(metrics_df.to_string(index=False, justify="right"))
            print_and_log("-"*30)
            # Add the DataFrame to the dictionary for plotting later
            plot_dict2[df_name][k] = metrics_df
            # Sleep for a few seconds to cool down the hardware
            time.sleep(3)
        print("="*150)
        # Plot the metrics on the top-k most important features
        # The X-axis is the number of features, with the kth feature as label
        # The Y-axis is the metrics, with three types of metrics shown in separate markers
        # The Y-axis also shows the metric scores for each feature
        # The metrics are AUC, F1, and Accuracy, which are saved in metrics_df
        # The legend is the type of the metric
        # Use tight layout
        plt.tight_layout()
        fig, ax = plt.subplots()
        # Set the title of the figure
        fig.suptitle(f"{df_name} Feature Selection ({model_name})")
        marker_map = {"AUC": "o", "F1": "s", "Accuracy": "^"}
        for metric_name in ["AUC", "F1", "Accuracy"]:
            # Create figure
            plt_name = f"{df_name}_top_features_metrics.png"
            x_data = [i for i in range(1, num_features+1)]
            # Append the feature name to each x_data
            x_data = ["0_random_noise_baseline"] + [f"{i}_{all_features_sorted[i-1]}" for i in x_data]
            model_idx = metrics_df["Model"].values.tolist().index(model_name)
            y_data = []
            y_err = []
            for metrics_df in plot_dict2[df_name].values():
                # Get the data for plotting
                y_data.append(metrics_df["Avg " + metric_name].values[model_idx])
                y_err.append(metrics_df["SE " + metric_name].values[model_idx])
            # Plot the data
            ax.errorbar(
                x_data,
                y_data,
                yerr=y_err,
                marker=marker_map[metric_name],
                label=metric_name + " (SE)",
                ms=2,
                capsize=2,
                barsabove=True,
                elinewidth=2
            )
            # Rotate the x-axis labels
        # Add feature scores
        y_data = [0]
        for feature, score in sorted(all_top_features[df_name][model_name].items(), key=lambda x: x[1], reverse=True):
            y_data.append(score)
        # Add 0.2 to y_data to make the plot look better
        y_data = [y + 0.2 for y in y_data]
        ax.plot(x_data, y_data, marker="x", label=f"Feature Score + 0.2 ({model_name})")
        # Set the marker size to smaller sizes so error bars can be seen
        # Make the x-axis labels smaller if len_x_data is too long
        x_tick_size = 10 if len(x_data) < 30 else 6
        plt.xticks(rotation=-45, ha="left", size=x_tick_size)
        # Add legend
        ax.legend(loc="best")
        # Enforce the range of the y-axis
        ax.set_ylim(0.2, 1.0)
        fig.set_size_inches(len(x_data) * 0.1 + 5, 9)
        # Show major gridlines
        ax.grid(True, which="major", linestyle="-")
        # Show minor gridlines
        ax.grid(True, which="minor", linestyle="--", alpha=0.2)
        plt.show()
        # Save the figure
        # Adjust plot size, let the width be proportional to the length of x-data
        plt_path = os.path.join(plot_dir, plt_name)
        fig.tight_layout()
        fig.savefig(plt_path, dpi=500)
        
        
plot_num_features_vs_metrics(all_top_features, model_name, target_column, metrics_df, plot_dict2, plot_dir)

: 

In [ ]:
# Backward step-wise feature selection

# Conduct experiments for determining the cumulative effect of the features on the model performance
# For each dataset, train the models on the top-k most important feature, then top-(k-1), top-(k-2), ...
def backward_feature_selection(all_top_features, model_name, target_column, metrics_df, plot_dict3, plot_dir, standardize, num_folds=3, impute_max_iter=30):
    
    for df_name in all_top_features:
        if not ("1.0" in df_name):
            continue
        # Create a DataFrame for the top-k most important feature
        # Sort all features by feature scores
        all_features_sorted = [f[0] for f in sorted(all_top_features[df_name][model_name].items(), key=lambda x: x[1], reverse=True)]
        num_features = len(all_features_sorted)
        df_all_features = dfs_std[df_name]
        df_all_features = df_all_features.dropna(axis=1, how="all")
        # If standardize is True, standardize the data
        if standardize:
            df_all_features = standardize_data(df_all_features)
        # Keep track of the bottom feature scores
        lowest_feature_scores = [0]
        lowest_feature_names = []
        for k in range(num_features, -1, -1):
            if k == 0:
                # Random noise baseline
                top_k_features = "random_noise_baseline"
                lowest_feature_names.append("0_random_noise_baseline")
                print_and_log(f"Processing {df_name} with top-{k} features out of {num_features}. This feature is random noise baseline.")
                df_top_k_features = df_all_features[[target_column]]
                noise_data = np.random.rand(len(df_top_k_features))
                df_top_k_features.insert(0, "0_random_noise_baseline", noise_data)
                assert len(df_top_k_features.columns) == 2
                lowest_feature_scores.append(0)
            elif k == num_features:
                top_k_features = f"{k}_all_features"
                lowest_feature_names.append(top_k_features)
                print_and_log(f"Processing {df_name} with top-{k} features out of {num_features}. This feature is all features.")
                df_top_k_features = df_all_features.copy()
                assert len(df_top_k_features.columns) == num_features + 1
            else:
                # Assert feature scores are sorted in descending order
                assert list(feature_scores.items()) == list(sorted(feature_scores.items(), key=lambda x: x[1], reverse=True))
                # Keep the top-k features and remove the rest, using feature_scores from previous iteration
                top_k_features = list(feature_scores.keys())[:-1]
                removed_feature = list(feature_scores.keys())[-1]
                assert len(top_k_features) == k
                lowest_feature_scores.append(feature_scores[removed_feature])
                lowest_feature_names.append(f"{k}_{removed_feature}")
                print_and_log(f"Processing {df_name} with top-{k} features out of {num_features}. The last removed feature was {removed_feature}.")
                # Create a DataFrame for the top-k most important features, and the target variable
                df_top_k_features = df_all_features[top_k_features + [target_column]]
                assert len(df_top_k_features.columns) == k + 1
                assert removed_feature not in df_top_k_features.columns
            # Print the number of missing values in df_top_k_features if any
            if df_top_k_features.isnull().values.any():
                print_and_log(f"{df_name} has {df_top_k_features.isnull().sum().sum()} missing values before imputing.")
            # Impute missing values in df_top_k_features using Random Forest
            df_top_k_features_imputed = impute_missing_value(df_top_k_features, target_column, max_iter=impute_max_iter)
            assert df_top_k_features_imputed.isnull().values.any() == False
            metrics_df, feature_scores = build_classifiers(df_top_k_features_imputed, df_name, num_folds=num_folds)
            # Keep feature scores from chosen model_name only, format is {"top-1st-feature": score1, "top-2nd-feature": score2, ...}
            feature_scores = feature_scores[model_name] 
            # Assert feature scores are sorted in descending order
            assert list(feature_scores.items()) == list(sorted(feature_scores.items(), key=lambda x: x[1], reverse=True))
            # print metrics_df, without index
            print_and_log("")
            print_and_log(metrics_df.to_string(index=False, justify="right"))
            print_and_log("-"*30)
            # Add the DataFrame to the dictionary for plotting later
            plot_dict3[df_name][k] = metrics_df
            # Sleep for a few seconds to cool down the hardware
            time.sleep(3)
        print("="*150)
        # Plot the metrics on the top-k most important features
        # The X-axis is the number of features, with the kth feature as label
        # The Y-axis is the metrics, with three types of metrics shown in separate markers
        # The Y-axis also shows the metric scores for each feature
        # The metrics are AUC, F1, and Accuracy, which are saved in metrics_df
        # The legend is the type of the metric
        # Use tight layout
        plt.tight_layout()
        fig, ax = plt.subplots()
        # Set the title of the figure
        fig.suptitle(f"{df_name} Feature Selection ({model_name})")
        marker_map = {"AUC": "o", "F1": "s", "Accuracy": "^"}
        for metric_name in ["AUC", "F1", "Accuracy"]:
            # Create figure
            plt_name = f"{df_name}_top_features_metrics.png"
            # x_data = [i for i in range(1, num_features+1)]
            # Append the feature name to each x_data
            # x_data = ["0_random_noise_baseline"] + [f"{i}_{all_features_sorted[i-1]}" for i in x_data]
            x_data = lowest_feature_names
            model_idx = metrics_df["Model"].values.tolist().index(model_name)
            y_data = []
            y_err = []
            for metrics_df in plot_dict3[df_name].values():
                # Get the data for plotting
                y_data.append(metrics_df["Avg " + metric_name].values[model_idx])
                y_err.append(metrics_df["SE " + metric_name].values[model_idx])
            # Plot the data
            ax.errorbar(
                x_data,
                y_data,
                yerr=y_err,
                marker=marker_map[metric_name],
                label=metric_name + " (SE)",
                ms=2,
                capsize=2,
                barsabove=True,
                elinewidth=2
            )

        ax.plot(x_data, lowest_feature_scores, marker="x", label=f"Removed Feature's Score ({model_name})")
        # Set the marker size to smaller sizes so error bars can be seen
        # Make the x-axis labels smaller if len_x_data is too long
        x_tick_size = 10 if len(x_data) < 30 else 6
        plt.xticks(rotation=-45, ha="left", size=x_tick_size)
        # Add legend
        ax.legend(loc="best")
        # Enforce the range of the y-axis
        ax.set_ylim(0, 1.0)
        fig.set_size_inches(len(x_data) * 0.1 + 5, 9)
        # Show major gridlines
        ax.grid(True, which="major", linestyle="-")
        # Show minor gridlines
        ax.grid(True, which="minor", linestyle="--", alpha=0.2)
        plt.show()
        # Save the figure
        # Adjust plot size, let the width be proportional to the length of x-data
        plt_path = os.path.join(plot_dir, plt_name)
        fig.tight_layout()
        fig.savefig(plt_path, dpi=500)
        

for standardize in True, False:
    dir_name = "plots_backward_selection_standardized" if standardize else "plots_backward_selection_not_standardized"
    plot_dir_backwards = os.path.join(dir_to_df, dir_name) 
    # Create the directory if it doesn't exist
    if not os.path.exists(plot_dir_backwards):
        os.makedirs(plot_dir_backwards)

    model_name = "Random Forest"
    target_column = "POS_did_the_patient_receive_pm"
    metrics_df = pd.DataFrame(columns=metrics_df_columns)
    # Create a dictionary for holding the DataFrames for plotting
    # Schema: {"Dataset1": {"TopK_feature": metrics_df1, "Top(K-1)_feature": metrics_df2, ...}, ...}
    plot_dict3 = defaultdict(OrderedDict)
    # Re-use the all_top_features dictionary for deciding the order of the features to use
    backward_feature_selection(all_top_features, model_name, target_column, metrics_df, plot_dict3, plot_dir_backwards, standardize=False, num_folds=5, impute_max_iter=100)

: 

: 